In [45]:
%reset -f

In [46]:
import pandas as pd
import seaborn as sns
from sklearn.manifold import TSNE
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import itertools

In [47]:
df = pd.read_csv("./land_mines.csv")
# df = df.replace({
#     "S":
#     {
#         1: "Dry and Sandy",
#         2: "Dry and Humus",
#         3: "Dry and Limy",
#         4: "Humid and Sandy",
#         5: "Humid and Humus",
#         6: "Humid and Limy"
#     },
#     "M":
#     {
#         1: "Null",
#         2: "Anti-tank",
#         3: "Anti-personnel",
#         4: "Booby Trapped Anti-personnel",
#         5: "M14 Anti-personnel"
#     }
# })
# #######################
# df = df[["V", "H", "M"]]

In [ ]:
plt.close('all')
df.hist()

In [ ]:
plt.close('all')
sns.heatmap(df.corr(), annot=True)

<div>За мапою кореляції видно низький вплив типу ґрунту S на значення напруги V.</div>
<div>Можливе його виключення:</div>
`df = df[["V", "H", "M"]]`

In [ ]:
plt.close('all')
sns.pairplot(df)

За парними графіками бачимо перетин множин "Є деяка міна" (M =! 1) та "Міни немає" (M == 1), тому залишимо тип ґрунту S у датасеті для виявлення його можливого затосування для зменшення невизначеності далі. За парними критеріями відсутня можливість знайти класифікаційну кореляцію.

In [ ]:
df_boolean = df.replace({"M": {1: False, 2: True, 3: True, 4: True, 5: True}})
plt.close('all')
sns.scatterplot(x="V", y="H", data=df_boolean, hue="M")

Приклад з напругою та висотою.

In [ ]:
plt.close('all')
sns.lmplot(x="V", y="S", data=df_boolean, hue="M", fit_reg=False)

Приклад з ґрунтом та напругою. 

In [ ]:
plt.close('all')
sns.scatterplot(x="V", y="M", data=df, hue="M")

Приклад з напругою та типом міни. Наявний невигідний перетин спостерігається при парній комбінації будь-яких параметрів.  

In [48]:
df_VHS = df.drop(["M"], axis=1)
tsne_features = TSNE().fit_transform(df_VHS)
df["x"] = tsne_features[:,0]
df["y"] = tsne_features[:,1]

In [49]:
tsne_features_3d = TSNE(n_components=3).fit_transform(df_VHS)
df["x_3d"] = tsne_features_3d[:,0]
df["y_3d"] = tsne_features_3d[:,1]
df["z_3d"] = tsne_features_3d[:,2]

In [ ]:
%matplotlib inline

plt.close('all')
sns.scatterplot(x='x', y='y', hue='M', data=df, alpha=0.5)
plt.show()

Вище описано неможливість точно вирішити проблему класифікацією за парними параметрами. Тут спостерігаємо успішне вирішення класифікації методами машинного навчання TSNE, що підтверджує необхідність використання щонайменш трьох з параметрів для умовного висновку. Це говорить про можливість розбиття процесу класифікації деревом рішень або послідовним перемиканням між значеннями параметрів серед нечітких множин.

In [ ]:
%matplotlib widget

plt.close('all')
fig = plt.figure()
fig.clear()
ax = fig.add_subplot(projection='3d')
ax.view_init(elev=25, azim=0, roll=0)
scatter = ax.scatter(df["x_3d"], df["y_3d"], df["z_3d"], c=df["M"])
plt.legend(*scatter.legend_elements())
plt.show()

Приклад спроможності класифікації методами машинного навчання TSNE у трьох вимірах.